In [1]:
import os
os.chdir("C:/Users/Sandhu/Desktop/hamoye data science")

In [2]:
# Import the necessary library
import pandas as pd
import numpy as np

In [3]:
# loading Our Dataset From Local Device
df = pd.read_csv('Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [4]:
# Check for dtypes
print(df.dtypes)

tau1     float64
tau2     float64
tau3     float64
tau4     float64
p1       float64
p2       float64
p3       float64
p4       float64
g1       float64
g2       float64
g3       float64
g4       float64
stab     float64
stabf     object
dtype: object


In [5]:
# Check for missing values
print(df.isnull().sum())

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64


In [6]:
# Delete the stab column
df = df.drop('stab', axis = 1)

# Importing an encoder
from sklearn.preprocessing import LabelEncoder

# Fit the encoder and transform stabf
label = LabelEncoder()
df['stabf'] = label.fit(df['stabf']).transform(df['stabf'])

# Importing a scaler for the data
from sklearn.preprocessing import StandardScaler

# Fit the scaler and transform the data
scaler = StandardScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Obtain x and y
x = scaled_df.drop('stabf', axis = 1)
y = df['stabf']

# Obtain the training (80%) and test data (20%)
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x , y, test_size = 0.2, random_state = 1)

# Importing Random Forest Classifier, setting the random state = 1
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state = 1)

# Fit the model
clf.fit(xtrain, ytrain)

# Predict on the test data
clf_pred = clf.predict(xtest)

# Obtain the accuracy
from sklearn.metrics import accuracy_score
clf_acc = accuracy_score(clf_pred, ytest)
print(format(clf_acc,".4f"))

0.9290


In [7]:
# Importing XGBoost Classifier, setting the random state = 1
import xgboost as xgb
model = xgb.XGBClassifier(random_state = 1)

# Fit the model
model.fit(xtrain, ytrain)

# Predict on the test data
model_pred = model.predict(xtest)

# Obtain the accuracy
model_acc = accuracy_score(model_pred, ytest)
print(model_acc)

C:\Users\Sandhu\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:38:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9455


In [8]:
# Importing LGBM Classifier, setting random state = 1
import lightgbm as gbm
lgbm = gbm.LGBMClassifier(random_state = 1)

# Fit the model
lgbm.fit(xtrain, ytrain)

# Predict on the test data
lgbm_pred = lgbm.predict(xtest)

# Obtain the accuracy
lgbm_acc = accuracy_score(lgbm_pred, ytest)
print(format(lgbm_acc,".4f"))

0.9400


In [9]:
# Importing Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
extra = ExtraTreesClassifier()

# Hyper-parameters
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

# Parameter grid
param_grid = {'n_estimators': n_estimators, 'min_samples_leaf': min_samples_leaf, 'min_samples_split': min_samples_split, 'max_features': max_features}

# Importing Random Search CV
from sklearn.model_selection import RandomizedSearchCV

# Random Search for hyper-parameter optimization
rscv = RandomizedSearchCV(extra, param_grid, cv = 5, n_iter = 10, scoring = 'accuracy', n_jobs = -1, verbose = 1, random_state = 1)

# Fit the model
rscv.fit(xtrain, ytrain)

# Obtain the best estimators
print(rscv.best_estimator_)
print(rscv.best_estimator_.min_samples_split)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
ExtraTreesClassifier(max_features=None, min_samples_leaf=8, n_estimators=1000)
2


In [10]:
# Taking the new optimal model
extclass = ExtraTreesClassifier(max_features = None, min_samples_leaf = 8, n_estimators = 1000, min_samples_split = 2, random_state = 1)

# Fit the model
extclass.fit(xtrain, ytrain)

# Predict on the test data
extclass_pred = extclass.predict(xtest)

# Obtain the accuracy
extclass_acc = accuracy_score(extclass_pred, ytest)
print(extclass_acc)

# Taking the initial Extra Trees Classifier with no hyper-parameter tuning
# Fit the model
extra.fit(xtrain, ytrain)

# Predict on the test data
extra_pred = extra.predict(xtest)

# Obtain the accuracy
extra_acc = accuracy_score(extra_pred, ytest)
print(extra_acc)

# extclass_pred < extra_pred
# The accuracy of the new optimal model is lower than the initial Extra Trees Classifier model with no hyper-parameter tuning

0.927
0.925


In [11]:
# Function to obtain the important features
def get_weights_df(model, feat, col_name):
  weights = pd.Series(model.feature_importances_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

extra.fit(xtrain, ytrain)

model_weights = get_weights_df(extra, xtrain, 'Feature_Importance')
model_weights.nsmallest

<bound method DataFrame.nsmallest of    Features  Feature_Importance
0        p1            0.039921
1        p3            0.040552
2        p4            0.040617
3        p2            0.041147
4        g1            0.089521
5        g4            0.094009
6        g2            0.094553
7        g3            0.095131
8      tau3            0.113164
9      tau1            0.115298
10     tau4            0.116295
11     tau2            0.119792>

####  Most important is tau2. Least important is p1.